Database ve tabloların oluşturulup kayıtların eklenmesi

In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT


# Veritabanı bağlantı bilgileri
credentials = {
    "dbname": "company",
    "user": "postgres",
    "password": "12345",
    "host": "localhost",
    "port": "5432"
}

# Veritabanı bağlantı bilgileri
credentials = {
    "dbname": "company",
    "user": "postgres",
    "password": "12345",
    "host": "localhost",
    "port": "5432"
}

# 1. Veritabanını oluştur (yoksa)
def create_database(creds):
    conn = psycopg2.connect(
        dbname="postgres",  # PostgreSQL'in varsayılan veritabanı
        user=creds["user"],
        password=creds["password"],
        host=creds["host"],
        port=creds["port"]
    )
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    try:
        cur.execute(f"CREATE DATABASE {creds['dbname']};")
        print(f"{creds['dbname']} veritabanı oluşturuldu.")
    except psycopg2.errors.DuplicateDatabase:
        print(f"{creds['dbname']} veritabanı zaten mevcut.")
    finally:
        cur.close()
        conn.close()

# 2. Tabloları oluştur ve veri ekle
def create_tables_and_insert_data(creds):
    conn = psycopg2.connect(**creds)
    cur = conn.cursor()

    # Mevcutsa silip yeniden oluşturmak istiyorsan önce drop komutu da eklenebilir:
    cur.execute("DROP TABLE IF EXISTS departments;")
    cur.execute("DROP TABLE IF EXISTS employees;")

    # departments tablosu
    cur.execute("""
        CREATE TABLE departments (
            emp_id INTEGER PRIMARY KEY,
            dept_name VARCHAR(50),
            dept_id INTEGER
        );
    """)

    # employees tablosu
    cur.execute("""
        CREATE TABLE employees (
            emp_id INTEGER PRIMARY KEY,
            first_name VARCHAR(50),
            last_name VARCHAR(50),
            salary INTEGER,
            job_title VARCHAR(100),
            gender VARCHAR(10),
            hire_date DATE
        );
    """)

    # Örnek veriler
    departments_data = [
        (17679, "Operations", 13),
        (26650, "Marketing", 14),
        (30840, "Operations", 13),
        (49823, "Technology", 12),
        (51821, "Operations", 13),
        (67323, "Marketing", 14),
        (71119, "Administrative", 11),
        (76589, "Operations", 13),
        (97927, "Technology", 12)
    ]

    employees_data = [
        (17679, "Robert", "Gilmore", 110000, "Operations Director", "Male", "2018-09-04"),
        (26650, "Elvis", "Ritter", 86000, "Sales Manager", "Male", "2017-11-24"),
        (30840, "David", "Barrow", 85000, "Data Scientist", "Male", "2019-12-02"),
        (49714, "Hugo", "Forester", 55000, "IT Support Specialist", "Male", "2019-11-22"),
        (51821, "Linda", "Foster", 95000, "Data Scientist", "Female", "2019-04-29"),
        (67323, "Lisa", "Wiener", 75000, "Business Analyst", "Female", "2018-08-09"),
        (70950, "Rodney", "Weaver", 87000, "Project Manager", "Male", "2018-12-20"),
        (71329, "Gayle", "Meyer", 77000, "HR Manager", "Female", "2019-06-28"),
        (76589, "Jason", "Christian", 99000, "Project Manager", "Male", "2019-01-21"),
        (97927, "Billie", "Lanning", 67000, "Web Developer", "Female", "2018-06-25")
    ]

    # Verileri ekle
    cur.executemany("""
        INSERT INTO departments (emp_id, dept_name, dept_id)
        VALUES (%s, %s, %s);
    """, departments_data)

    cur.executemany("""
        INSERT INTO employees (emp_id, first_name, last_name, salary, job_title, gender, hire_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    """, employees_data)

    conn.commit()
    cur.close()
    conn.close()
    print("Tablolar oluşturuldu ve veriler eklendi.")

# Hepsini çalıştır
create_database(credentials)
create_tables_and_insert_data(credentials)


company veritabanı zaten mevcut.
Tablolar oluşturuldu ve veriler eklendi.


In [2]:
def create_database(creds):
    conn = psycopg2.connect(
        dbname="postgres",
        user=creds["user"],
        password=creds["password"],
        host=creds["host"],
        port=creds["port"]
    )
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    try:
        cur.execute(f"CREATE DATABASE {creds['dbname']};")
        print(f"{creds['dbname']} veritabanı oluşturuldu.")
    except psycopg2.errors.DuplicateDatabase:
        print(f"{creds['dbname']} veritabanı zaten mevcut.")
    finally:
        cur.close()
        conn.close()

create_database(credentials)

company veritabanı zaten mevcut.


In [3]:
def run_query(query, creds):
    conn = psycopg2.connect(**creds)
    cur = conn.cursor()
    cur.execute(query)
    results = cur.fetchall()
    for row in results:
        print(row)
    cur.close()
    conn.close()

1 - Find the employees who get paid more than Rodney Weaver.

In [4]:
query = """
SELECT first_name, last_name, salary
FROM employees
WHERE salary > (
    SELECT salary FROM employees
    WHERE first_name = 'Rodney' AND last_name = 'Weaver'
);
"""

run_query(query, credentials)

('Robert', 'Gilmore', 110000)
('Linda', 'Foster', 95000)
('Jason', 'Christian', 99000)


2 - Find the average, min and max salaries

In [5]:
query2 = """
SELECT AVG(salary) AS avg_salary,
       MIN(salary) AS min_salary,
       MAX(salary) AS max_salary
FROM employees;
"""
run_query(query2, credentials)

(Decimal('83600.000000000000'), 55000, 110000)


3 - Find the employees whose salary is more than 8700. Our query should return first name, last name, and salary info of the employees.

In [6]:
query3 = """
SELECT first_name, last_name, salary
FROM employees
WHERE salary > 8700;
"""
run_query(query3, credentials)


('Robert', 'Gilmore', 110000)
('Elvis', 'Ritter', 86000)
('David', 'Barrow', 85000)
('Hugo', 'Forester', 55000)
('Linda', 'Foster', 95000)
('Lisa', 'Wiener', 75000)
('Rodney', 'Weaver', 87000)
('Gayle', 'Meyer', 77000)
('Jason', 'Christian', 99000)
('Billie', 'Lanning', 67000)


4 - Find the employees (first name, last name from employees table) who work under the Operations department (departments table). Our query should return first name and last name info.

In [ ]:
query4 = """
SELECT e.first_name, e.last_name
FROM employees e
JOIN departments d ON e.emp_id = d.emp_id
WHERE d.dept_name = 'Operations';
"""
run_query(query4, credentials)
#31354422222222222

('Robert', 'Gilmore')
('David', 'Barrow')
('Linda', 'Foster')
('Jason', 'Christian')


5 - Find the employees (first name, last name from employees table) who work under the Technology department (departments table). Our query should return first name and last name info.

In [8]:
query5 = """
SELECT e.first_name, e.last_name
FROM employees e
JOIN departments d ON e.emp_id = d.emp_id
WHERE d.dept_name = 'Technology';
"""
run_query(query5, credentials)

('Billie', 'Lanning')


6 - Find the average salary of female employees.

In [9]:
query6 = """
SELECT AVG(salary) AS avg_female_salary
FROM employees
WHERE gender = 'Female';
"""
run_query(query6, credentials)

(Decimal('78500.000000000000'),)


7 - Find the average salaries of each department.

In [10]:
query7 = """
SELECT d.dept_name, AVG(e.salary) AS avg_salary
FROM employees e
JOIN departments d ON e.emp_id = d.emp_id
GROUP BY d.dept_name;
"""
run_query(query7, credentials)

('Technology', Decimal('67000.000000000000'))
('Marketing', Decimal('80500.000000000000'))
('Operations', Decimal('97250.000000000000'))


8 - Find the oldest and newest employees.

In [11]:
query8 = """
SELECT first_name, last_name, hire_date
FROM employees
WHERE hire_date = (SELECT MIN(hire_date) FROM employees)
   OR hire_date = (SELECT MAX(hire_date) FROM employees);
"""
run_query(query8, credentials)

('Elvis', 'Ritter', datetime.date(2017, 11, 24))
('David', 'Barrow', datetime.date(2019, 12, 2))


9 - Find the hiring date and department of the highest paid employee

In [12]:
query9 = """
SELECT e.hire_date, d.dept_name
FROM employees e
JOIN departments d ON e.emp_id = d.emp_id
WHERE e.salary = (SELECT MAX(salary) FROM employees);
"""
run_query(query9, credentials)


(datetime.date(2018, 9, 4), 'Operations')


10 - Find the hiring date and department of the lowest paid employee

In [13]:
query10 = """
SELECT e.hire_date, d.dept_name
FROM employees e
JOIN departments d ON e.emp_id = d.emp_id
WHERE e.salary = (SELECT MIN(salary) FROM employees);
"""
run_query(query10, credentials)